In [2]:
import pandas as pd

PyPDF2 Ventajas:

Fácil de usar para tareas básicas de lectura y manipulación de archivos PDF.

Buena documentación y comunidad activa.

Inconvenientes:

La extracción de texto no siempre es precisa, especialmente en documentos PDF complejos.

No es la mejor opción para extraer imágenes o tablas

In [3]:
from PyPDF2 import PdfReader

with open('RSA - AUTONOMOS-AS_ALBERTO BLANCO CORVINOS - GESTORINFORMATICO.COM.pdf', 'rb') as file:
    reader = PdfReader(file)
    page = reader.pages[0]
    text = page.extract_text()
    print(text)

RSA - AUTONOMOS-AS
Empresa evaluada
ALBERTO BLANCO CORVINOS -
GESTORINFORMATICO.COM
CIF:18050059G
C/Felipe Coscolla nº:16, 1ºD
22004 - Huesca
Huesca
DATOS DE LA EMPRESA
01 - Denominación de la organización
ALBERTO BLANCO CORVINOS cómo autónomo sin empleados, marca comercial
gestorinformatico.com
02 - Dirección - CP - POBLACION
C/ Felipe Coscolla 16 1ºDrcha, 22004, HUESCA (Huesca)
03 - Sector. Actividad
Gestión de nuevas tecnologías en la Pyme (asociaciones, Pymes y autónomos), desarrollo
web, mantenimiento Hardware/Software & Redes informáticas, Gestión Cloud Computing,
Hosting & RSS
04 - Año comienzo actividad.
Enero de 2019
05 - Persona de contacto
ALBERTO BLANCO CORVINOS
677461744
06 - e-mail contacto:
albertoblancoonline@gmail.com
07.- Página web
gestorinformatico.com
albertoblancor.com
GENERAL
Tendencias Globales
1- Considera que su organización tiene en cuenta o se ve afectada, directa o
indirectamente, por temas globales y generales como, por ejemplo:
- Económicos (por ejemplo, 

In [4]:
import fitz  # PyMuPDF

def extraer_datos_empresa(pdf_path):
    datos_empresa = {}
    
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
            # Puedes ajustar los siguientes extractores según el formato específico del PDF
            if "DATOS DE LA EMPRESA" in text:
                datos_empresa['nombre_organizacion'] = text.split('01 - Denominación de la organización\n')[1].split('\n')[0].strip()
                datos_empresa['direccion'] = text.split('02 - Dirección - CP - POBLACION\n')[1].split('\n')[0].strip()
                datos_empresa['sector'] = text.split('03 - Sector. Actividad\n')[1].split('\n')[0].strip()
                datos_empresa['ano_comienzo_actividad'] = text.split('04 - Año comienzo actividad.\n')[1].split('\n')[0].strip()
                datos_empresa['persona_contacto'] = text.split('05 - Persona de contacto\n')[1].split('\n')[0].strip()
                datos_empresa['email_contacto'] = text.split('06 - e-mail contacto:\n')[1].split('\n')[0].strip()
                datos_empresa['pagina_web'] = text.split('07.- Página web\n')[1].split('\n')[0].strip()
                #datos_empresa['nivel_priorizacion_medio_ambiente'] = text.split('Trabajar para proteger y mejorar el medio ambiente')
                break           
    return datos_empresa


# Ejemplo de uso
pdf_path = 'RSA - AUTONOMOS-AS_ALBERTO BLANCO CORVINOS - GESTORINFORMATICO.COM.pdf'
datos_empresa = extraer_datos_empresa(pdf_path)
df = pd.DataFrame([datos_empresa])


In [5]:
df.T

,0
nombre_organizacion,ALBERTO BLANCO CORVINOS cómo autónomo sin empl...
direccion,"C/ Felipe Coscolla 16 1ºDrcha, 22004, HUESCA (..."
sector,Gestión de nuevas tecnologías en la Pyme (asoc...
ano_comienzo_actividad,Enero de 2019
persona_contacto,ALBERTO BLANCO CORVINOS
email_contacto,albertoblancoonline@gmail.com
pagina_web,gestorinformatico.com


In [6]:
# Tenemos que considerar que los formularios de diferentes tipo de empresas cambian los numeros de los titulos y con el primer metodo solo sale la primera linea de texto. 

import fitz  # PyMuPDF
import re

def extraer_datos(pdf_path):
    datos = {}
    
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf[page_num]
            text = page.get_text()
# Patrón tipo empresa:
            patron_tipo = r"\n*RSA - (.*?)\n*Empresa evaluada"
            match_tipo = re.search(patron_tipo, text, re.DOTALL)
            if match_tipo:
                datos['tipo_organizacion'] = match_tipo.group(1).replace("\n", " ").strip()
# Patrón datos nombre-direccion-sector-año comienzo-contacto:
            patron_nombre = r"\d{2} - Denominación de la organización\n(.*?)\n\d{2} - Dirección - CP - POBLACION"
            match_nombre = re.search(patron_nombre, text, re.DOTALL | re.IGNORECASE)
            if match_nombre:
                datos['nombre_organizacion'] = match_nombre.group(1).replace("\n", " ").strip()
            
            patron_direccion = r"\d{2} - Dirección - CP - POBLACION\n(.*?)\n\d{2} - Sector. Actividad"
            match_direccion = re.search(patron_direccion, text, re.DOTALL| re.IGNORECASE)
            if match_direccion:
                datos['direccion'] = match_direccion.group(1).replace("\n", " ").strip()
            
            patron_sector = r'\d{2} - Sector. Actividad\n(.*?)\n\d{2} - Año comienzo actividad'
            match_sector = re.search(patron_sector, text, re.DOTALL)
            if match_sector:
                datos['sector_actividad'] = match_sector.group(1).replace("\n", " ").strip()

            patron_ano_comienzo = r'\d{2} - Año comienzo actividad.\n(.*?)\n\d{2} - Persona de contacto'
            match_ano_comienzo = re.search(patron_ano_comienzo, text, re.DOTALL)
            if match_ano_comienzo:
                datos['ano_comienzo_actividad'] = match_ano_comienzo.group(1).replace("\n", " ").strip()

            patron_persona_contacto = r'\d{2} - Persona de contacto\n(.*?)\n\d{2} - e-mail contacto:'
            match_persona_contacto = re.search(patron_persona_contacto, text, re.DOTALL)
            if match_persona_contacto:
                datos['persona_contacto'] = match_persona_contacto.group(1).replace("\n", " ").strip()

            patron_email_contacto = r'\d{2} - e-mail contacto:\n(.*?)\n\d{2}.- Página web'
            match_email_contacto = re.search(patron_email_contacto, text, re.DOTALL)
            if match_email_contacto:
                datos['email_contacto'] = match_email_contacto.group(1).replace("\n", " ").strip()

            patron_pagina_web = r'\d{2}.- Página web\n(.*?)\n*GENERAL'
            match_pagina_web = re.search(patron_pagina_web, text, re.DOTALL)
            if match_pagina_web:
                datos['pagina_web'] = match_pagina_web.group(1).replace("\n", " ").strip()
# Patrón Impacto ambiental de su actividad:
            patron_impacto_actividad = r'\nambiental en el desarrollo de la actividad\?\n(.*?)\n\d+.1. Información adicional.'
            match_impacto_actividad = re.search(patron_impacto_actividad, text, re.DOTALL)
            if match_impacto_actividad:
                datos['impacto_actividad'] = match_impacto_actividad.group(1).replace("\n", " ").strip()


# Patrón para buscar la importancia de 1 a 10 que le da a "Trabajar para proteger y mejorar el medio ambiente"
            patron_prioridad_medioambiental = r"(\d+) - Trabajar para proteger y mejorar el medio ambiente"
            match_prioridad_ma = re.search(patron_prioridad_medioambiental, text, re.DOTALL)
            if match_prioridad_ma:
                datos['prioridad mediambiental'] = match_prioridad_ma.group(1)
# Patrón para buscar el áreas de mejora
            patron_mejora = r"\d+ - ¿Podría enunciar un área de mejora\?\n(.*?)\nAgenda 2030 y Objetivos de Desarrollo Sostenible"
            match_mejora = re.search(patron_mejora, text, re.DOTALL)
            if match_mejora:
                datos['mejora'] = match_mejora.group(1).replace("\n", " ").strip()
    return datos

# Ejemplo de uso
pdf_path = 'RSA - AUTONOMOS-AS_ALBERTO BLANCO CORVINOS - GESTORINFORMATICO.COM.pdf'
datos = extraer_datos(pdf_path)
df2 = pd.DataFrame([datos])



In [9]:
print(df2.T.to_string())



                                                                                                                                                                                                                0
tipo_organizacion                                                                                                                                                                                    AUTONOMOS-AS
nombre_organizacion                                                                                                    ALBERTO BLANCO CORVINOS cómo autónomo sin empleados, marca comercial gestorinformatico.com
direccion                                                                                                                                                   C/ Felipe Coscolla 16 1ºDrcha, 22004, HUESCA (Huesca)
sector_actividad         Gestión de nuevas tecnologías en la Pyme (asociaciones, Pymes y autónomos), desarrollo web, mantenimiento Hardware/Software & Redes inf

In [11]:
pdf_path_3 = 'RSA - AUTONOMOS-AS_ALBERTO BLANCO CORVINOS - GESTORINFORMATICO.COM.pdf'
datos = extraer_datos(pdf_path_3)
df3 = pd.DataFrame([datos])

In [12]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   tipo_organizacion        1 non-null      object
 1   nombre_organizacion      1 non-null      object
 2   direccion                1 non-null      object
 3   sector_actividad         1 non-null      object
 4   ano_comienzo_actividad   1 non-null      object
 5   persona_contacto         1 non-null      object
 6   email_contacto           1 non-null      object
 7   pagina_web               1 non-null      object
 8   impacto_actividad        1 non-null      object
 9   prioridad mediambiental  1 non-null      object
 10  mejora                   1 non-null      object
dtypes: object(11)
memory usage: 220.0+ bytes


| Variable                  | Tipo   |Descripción|
|---------------------------|--------|-|
| tipo_organizacion         | object |Autonomo (120), Pequeña empresa (850), Mediana empresa (124), Gran empresa (113), Empresa publica (18), Entidades(301), Ong(47), ojo el formulario es el mismo en contenido pero se han detectado diferencias, necesario ajustes |
| nombre_organizacion       | object |Anonimizaremos por Tipo+numero|
| direccion                 | object |Nos servira para geolocalizar en mapa|
| sector_actividad          | object |Analizar, variable importante para agrupar, esta en lenguaje natural, habra que standarizarla de alguna forma|
| ano_comienzo_actividad    | object |Convertir a temporal, consideramos puede ser relevante|
| persona_contacto          | object |Anonimizar, se usara en la fase de contacto|
| email_contacto            | object |Anonimizar, seria interesante analizar codigo necesario para envio masivo email contacto|
| pagina_web                | object |Solo util para profundizar departamento comercial para adecuar oferta de servicios y conocer al cliente.|
| impacto_actividad         | object |Es lenguaje natural y recoge la valoración que da el cliente al impacto de su actividad en el medioambiente, puede ser interesante o no|
| prioridad mediambiental   | object |En un primer momento valoro solo indicar de 1 a 10 la importancia que le da a medioambiente, pero barajo la opcion de extender y recoger la escala de priorizacion de las 10 propuesta, ya que podria aportar matices del perfil de la entidad
|||Emplear prácticas empresariales abiertas y transparentes
|||Gestionar con ejemplaridad
|||Escuchar las necesidades y expectativas de sus grupos de interés
|||Poner en práctica medidas responsables en la gestión de Recursos Humanos
|||Ofrecer productos y servicios de calidad
|||Capacitación y desarrollo profesional
|||Disponer de presupuesto
|||Trabajar para proteger y mejorar el medio ambiente
|||Crear Programas con impacto positivo en la Comunicad Local
|||Ser un negocio rentable y sostenible en el tiempo|
| mejora                    | object |Es lenguaje natural y creo seria interesante detectar palabras claves que releven necesidades a cubrir en medioambiente o en otras areas que detecten nichos de mercado futuros para la entidad|
